In [1]:
import requests
from bs4 import BeautifulSoup 
from datetime import datetime                            
import pandas as pd 
import time 
import urllib.request 
from selenium.webdriver import Chrome
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import time
    

In [2]:
company_list = ['002840', '107590', '134380', '003650', '084990', '052260', '003610', '001340', '014580', '035150',]

In [3]:
def stock_crawler(code):
    
    browser  = Chrome()
    browser.maximize_window()
    
    name = code
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    
    
    browser.get(base_url)
    time.sleep(2)
   
    browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))
    time.sleep(2)
    
    browser.find_elements_by_xpath('//*[@class="schtab"][1]/tbody/tr/td[3]')[0].click()
    time.sleep(2)

    html0 = browser.page_source
    html1 = BeautifulSoup(html0,'html.parser')
    
    #기업명 뽑기
    title0 = html1.find('head').find('title').text
    print(title0.split('-')[-1])
    
    html22 = html1.find('table',{'class':'gHead01 all-width','summary':'주요재무정보를 제공합니다.'})
    
    #date scrapy
    thead0 = html22.find('thead')
    tr0 = thead0.find_all('tr')[1]
    th0 = tr0.find_all('th')
    
    date = []
    for i in range(len(th0)):
        date.append(''.join(re.findall('[0-9/]',th0[i].text)))
    
    #columns scrapy
    tbody0 = html22.find('tbody')
    tr0 = tbody0.find_all('tr')
    
    col = []
    for i in range(len(tr0)):

        if '\xa0' in tr0[i].find('th').text:
            tx = re.sub('\xa0','',tr0[i].find('th').text)
        else:
            tx = tr0[i].find('th').text

        col.append(tx)
    
    #main text scrapy
    td = []
    for i in range(len(tr0)):
        td0 = tr0[i].find_all('td')
        td1 = []
        for j in range(len(td0)):
            if td0[j].text == '':
                td1.append('0')
            else:
                td1.append(td0[j].text)

        td.append(td1)
    
    td2 = list(map(list,zip(*td)))
    
    ###
    
    multiple_index = []    # 빈 리스트 생성
 
    for i in range(len(date)):
      
        multiple_index.append(code)
    
    
    
    ####
    df = pd.DataFrame(td2,columns = col,index = [multiple_index, date])
    
    ###5번째 행까지만 살리기
    df.drop(df.index[5:], inplace=True)
    time.sleep(1)
    
    
    return df

In [4]:
df_first_row = stock_crawler('005930')

<ipython-input-3-f8e7fdd6f8db>:13: DeprecationWarning: use driver.switch_to.frame instead
  browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))


 기업개요(삼성전자)


In [5]:
df_first_row

매출액     영업이익 영업이익(발표기준) 세전계속사업이익    당기순이익 당기순이익(지배)  \
005930 2016/12  2,018,667  292,407    292,407  307,137  227,261   224,157   
       2017/12  2,395,754  536,450    536,450  561,960  421,867   413,446   
       2018/12  2,437,714  588,867    588,867  611,600  443,449   438,909   
       2019/12  2,304,009  277,685    277,685  304,322  217,389   215,051   
       2020/12  2,368,070  359,939    359,939  363,451  264,078   260,908   

               당기순이익(비지배)       자산총계       부채총계       자본총계  ...   부채비율  \
005930 2016/12      3,104  2,621,743    692,113  1,929,630  ...  35.87   
       2017/12      8,422  3,017,521    872,607  2,144,914  ...  40.68   
       2018/12      4,540  3,393,572    916,041  2,477,532  ...  36.97   
       2019/12      2,338  3,525,645    896,841  2,628,804  ...  34.12   
       2020/12      3,170  3,782,357  1,022,877  2,759,480  ...  37.07   

                    자본유보율 EPS(원) PER(배)  BPS(원) PBR(배) 현금DPS(원) 현금배당수익률  \
005930 2016/12  22,004.14  2,735  13.18  26,636   1.35      570    1.58   
       2017/12  24,536.12  5,421   9.40  30,427   1.67      850    1.67   
       2018/12  27,531.92  6,024   6.42  35,342   1.09    1,416    3.66   
       2019/12  28,856.02  3,166  17.63  37,528   1.49    1,416    2.54   
       2020/12  30,692.79  3,841  21.09  39,406   2.06    2,994    3.70   

               현금배당성향(%)     발행주식수(보통주)  
005930 2016/12     17.81  7,033,966,850  
       2017/12     14.09  6,454,924,700  
       2018/12     21.92  5,969,782,550  
       2019/12     44.73  5,969,782,550  
       2020/12     77.95  5,969,782,550  

[5 rows x 33 columns]

In [6]:
for i in company_list :
    df_new = stock_crawler(i)
    df_first_row = pd.concat([df_first_row, df_new])
    
    
    
    ##
    if i == company_list[5] :
        df_first_row.to_excel('tobinq.xlsx')
        time.sleep(3)
        
    
    ##
    

<ipython-input-3-f8e7fdd6f8db>:13: DeprecationWarning: use driver.switch_to.frame instead
  browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))


 기업개요(미원상사)
 기업개요(미원홀딩스)
 기업개요(미원화학)
 기업개요(미창석유)
 기업개요(헬릭스미스)
 기업개요(현대바이오랜드)
 기업개요(방림)
 기업개요(백광산업)
 기업개요(태경비케이)
 기업개요(백산)


In [7]:
df_all = df_first_row
df_all.to_excel('tobinq.xlsx')